In [21]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from copy import deepcopy

import scipy

ModuleNotFoundError: No module named 'scipy'

In [13]:
numberOfimages=1
for image in range(numberOfimages):
    if image==0:
        img1=cv2.imread('./images/000'+str(image).zfill(3)+'.png')
        img2=cv2.imread('./images/000'+str(image+1).zfill(3)+'.png')
    else:
        img1=deepcopy(img2)
        img2=cv2.imread('./images/000'+str(image+1).zfill(3)+'.png')  
    sift = cv2.xfeatures2d.SIFT_create()


error: OpenCV(3.4.2) /tmp/build/80754af9/opencv-suite_1535558553474/work/modules/highgui/src/window.cpp:615: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Carbon support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvNamedWindow'
